In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy
from IPython.display import Image
from qutip import *
from math import sqrt


from qutip.qip.device import Processor
from qutip_qip.device import DispersiveCavityQED
from qutip.qip.pulse import Pulse
from qutip.operators import sigmaz, sigmay, sigmax, destroy
from qutip.states import basis
from qutip.metrics import fidelity
from qutip.qip.operations import rx, ry, rz, hadamard_transform, cphase
from qutip_qip.circuit import QubitCircuit, Gate
# from qutip.qip.operations import cnot, snot

The ramsey simulation builds on source code to do a ramsey measurement of $T_2$ for a qubit. The changes to the source code are in the model used: all examples use the LinearSpinChain model, processor, and compiler. Our code uses the DispersiveCavityQED processor, which uses the Tavis-Cummings Hamiltonian as a model. The relevant source code is at the following links:
- https://qutip.org/docs/latest/guide/qip/qip-processor.html 
- https://arxiv.org/pdf/2105.09902.pdf

### Setting the System Constants + Operators
All are in units of GHz. No experimental baseline was used to set the constants, however, this can be easily done.

In [4]:
# qubit resonance frequency
wq = 3.04 * 2 * np.pi
# cavity resonance frequency
wc = 3.00 * 2 * np.pi
# coupling
g = 0.08 * 2 * np.pi
# frequency of pulses
f = 0.5
# amplitude of pulses
amp = 0.1
# number of levels in cavity
num_c_levels = 20
# number of qubits coupled to cavity
num_qubits = 1
# define the dephasing rate to feed into the model
T2 = 10/f

### Defining the Processor and Drift
In order to create the pulse level simulation, we define the model, processor, and compiler. DispersiveCavityQED is all of those at once: we call it the processor because the processor uses the CavityQEDModel and the CavityQEDCompiler. The drift term and extra control in the Hamiltonian are z-spin drift and x-spin including the z-drift factor.

In [5]:
# define the processor, model, and compiler with x-pulse control
processor = DispersiveCavityQED(num_qubits=num_qubits, num_levels = num_c_levels, sx=amp/2, w0=wc, g=g, t2=T2)
# define the z-drift term
ham_idle= 2*np.pi * tensor(qeye(num_c_levels),sigmaz())/2 * f
# define the x-z combined control
resonant_sx = 2*np.pi * tensor(qeye(num_c_levels),sigmax())- ham_idle / (amp/2)

# add to the model. Targets are the qubit and cavity
processor.add_drift(ham_idle, targets = [0,1])
processor.add_control(resonant_sx , targets = [0,1] , label ="sx0")


/opt/anaconda3/envs/qutip-env/lib/python3.10/site-packages/qutip_qip/device/cavityqed.py:334: UserWarning: Not in the dispersive regime
  warnings.warn("Not in the dispersive regime")
/opt/anaconda3/envs/qutip-env/lib/python3.10/site-packages/qutip_qip/device/cavityqed.py:337: UserWarning: The rotating-wave approximation might not be valid.
  warnings.warn(


### Create a Circuit and Experiment
We create the circuit to run pulese into and add it to the processor. We do this over time t. The RX gate applies a pulse of specified bloch sphere rotation value, and IDLE makes the qubit "wait" for time t until the next pulse. We can then measure the final state. 

In [7]:
def ramsey(t,proc):
    qc = QubitCircuit(1)
    qc.add_gate("RX", targets=[0], arg_value =np.pi/2) # move the bloch vector to equator
    qc.add_gate("IDLE", targets=[0], arg_value = t )
    qc.add_gate("RX", targets=[0], arg_value =np.pi/2)
    
    # add the circuit to the processor so it can run pulses
    proc.load_circuit(qc)
    # specify the operators to measure when getting expectation values
    e_ops = [tensor(qeye(num_c_levels), sigmaz())]
    # run the processor
    result = processor.run_state(init_state=tensor(basis(num_c_levels,0), basis(2,0)), e_ops=e_ops, options=Options(nsteps=10000))
    return result.expect[0][-1]

### Run the Experiment
Run the ramsey function for the specified times and get the peaks of the oscillations to fit a decay curve to

In [9]:
num_samples=500
idle_tlist = np.linspace(0., 50., num_samples)
measurements = np.asarray([ramsey(t, processor) for t in idle_tlist])

rx_gate_time = 1/4/amp/np.pi/2
total_time = 2*rx_gate_time + idle_tlist[-1]
tlist = np.linspace(0., total_time, num_samples)

peak_ind = scipy.signal.find_peaks(measurements)[0]
peak_signal = measurements[peak_ind]
peak_times = idle_tlist[peak_ind]

### Graph the Result and Fit the Decay Fn
The result of the fitting to the osc

In [ ]:
decay_func = lambda t, t2, f0, w0: f0 * np.exp(-t/t2) + w0
(t2_fit,f0_fit, w0_fit),_= scipy.optimize.curve_fit(decay_func, peak_times[1:], peak_signal[1:], maxfev=500000)
print("T2:", T2)
print("Fitted T2:", t2_fit)

fig, ax = plt.subplots(figsize = (5, 3), dpi=100)
ax.plot(idle_tlist[:], measurements[:], '-', label="Simulation", color="slategray")
ax.plot(idle_tlist, decay_func(idle_tlist, t2_fit,f0_fit, w0_fit), '--', label="Theory", color="slategray")
ax.scatter(peak_times, peak_signal, color="r")
ax.set_xlabel(f"Idling time t, ns \n T2={T2}, fitted T2={round(t2_fit, 3)}")
ax.set_ylabel("Ramsey signal", labelpad=2)
ax.set_ylim((ax.get_ylim()[0], ax.get_ylim()[1]))
ax.set_position([0.18, 0.2, 0.75, 0.75])
ax.grid()
fig
